In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:

from src.preparation import fetch_dataset
from src.modules.visualizer import do_heatmap
dataset = fetch_dataset('dataset_all_slim.pkl')



In [3]:
dataset.head()


,Filename,LabelVector,Begin,End,Sample,CEPST,SPECT,MFCC,CWT,ZCR
0,rec2018-02-02_01h34m08.386s.wav,Noise,52279,58753,"[[-79, -63, -30, 6, 14, 23, 27, 18, 17, 10, -1...","[[[1.693221, 1.775855, 1.741249, 1.616202, 2.1...","[[[0.011621, 0.010184, 0.011724, 0.015311, 0.0...","[[[-18.136064, -15.67883, -13.065683, -11.4263...","[[[0.196308, 0.210414, 0.217209, 0.187533, 0.1...","[[0.976517, 0.988258, 0.972603, 0.95499, 0.951..."
1,rec2018-02-02_01h34m08.386s.wav,Noise,44530,52412,"[[-3, 3, 4, 10, 7, 4, -1, -14, -8, 7, 2, -5, -...","[[[2.140195, 1.835786, 1.860486, 1.820666, 1.7...","[[[0.006132, 0.004942, 0.003436, 0.005528, 0.0...","[[[-3.256531, -4.392584, -5.202826, -5.233054,...","[[[0.072738, 0.071596, 0.071802, 0.073953, 0.0...","[[0.8591, 0.835616, 0.833659, 0.841487, 0.8767..."
2,rec2018-02-02_01h34m08.386s.wav,Noise,2670,8888,"[[-138, -164, -105, 29, 144, 172, 240, 182, 46...","[[[1.076635, 1.698971, 2.232763, 1.433347, 0.6...","[[[0.01812, 0.00675, 0.00852, 0.014756, 0.0331...","[[[-11.771419, -14.452424, -20.437652, -27.309...","[[[0.678713, 0.447045, 0.399548, 0.39986, 0.57...","[[0.992172, 0.992172, 0.992172, 1.0, 1.0, 1.0,..."
3,rec2018-02-02_01h34m08.386s.wav,Noise,15018,25089,"[[-6, -12, -8, -1, -6, 7, 19, 16, 4, 2, 13, 11...","[[[1.609493, 1.033311, 1.006966, 1.05215, 0.83...","[[[0.014096, 0.018861, 0.024633, 0.011562, 0.0...","[[[-22.452866, -24.279197, -23.800461, -22.455...","[[[0.105505, 0.156634, 0.183518, 0.18604, 0.18...","[[0.988258, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
4,rec2018-01-23_10h59m20.895s.wav,Exhale,3454,15745,"[[194, 117, 14, -74, -118, -167, -190, -102, 3...","[[[1.643628, 0.942107, 0.530973, 0.273131, 0.2...","[[[0.011266, 0.015878, 0.025697, 0.043823, 0.0...","[[[-33.276069, -29.523152, -27.258144, -27.167...","[[[0.192724, 0.851869, 2.405978, 5.085328, 7.8...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."


In [4]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import LSTM

In [5]:
# This is the size of the window that is fed into the DNN
window_size = 15
# The number of the features present in the dataset
num_of_features = 42
# Number of distinct labels in the output
label_length = 4
# Hyperparameter that defines the number of samples to work through
# before updating the internal model parameters.
batch_size = 25
# Epochs
ep = 100

In [6]:
# Create a model
model = Sequential()
model.add(LSTM(100, return_sequences=False, input_shape=(window_size, num_of_features)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(label_length, activation='softmax'))
model.compile(loss='binary_crossentropy',
                   optimizer='rmsprop',
                   metrics=['accuracy',])


In [20]:
from src.processing import make_tf_dataset, split_train_test

df_train, df_test = split_train_test(dataset= dataset, percentage= 0.8)

data_train = make_tf_dataset(df_train, window_size, num_of_features, label_length ).batch(batch_size)
data_test = make_tf_dataset(df_test ,window_size, num_of_features, label_length).batch(batch_size)


In [26]:
df_train.shape

(927, 10)

In [27]:
df_test.shape

(231, 10)

In [35]:
dataset = fetch_dataset('dataset_a1_f1_slim.pkl')
data_train = make_tf_dataset(dataset, window_size, num_of_features, label_length ).batch(batch_size)



model.fit(data_train, epochs=ep)


Epoch 1/100
2623/2623 [==============================] - 80s 31ms/step - loss: 0.1000 - accuracy: 0.9696
Epoch 2/100
2623/2623 [==============================] - 77s 29ms/step - loss: 0.0993 - accuracy: 0.9706
Epoch 3/100
2623/2623 [==============================] - 77s 29ms/step - loss: 0.0993 - accuracy: 0.9705
Epoch 4/100
2623/2623 [==============================] - 77s 29ms/step - loss: 0.1017 - accuracy: 0.9704
Epoch 5/100
2623/2623 [==============================] - 77s 29ms/step - loss: 0.0989 - accuracy: 0.9711
Epoch 6/100
2623/2623 [==============================] - 77s 29ms/step - loss: 0.1007 - accuracy: 0.9706
Epoch 7/100
2623/2623 [==============================] - 78s 30ms/step - loss: 0.0979 - accuracy: 0.9719
Epoch 8/100
2623/2623 [==============================] - 79s 30ms/step - loss: 0.0959 - accuracy: 0.9725
Epoch 9/100
2623/2623 [==============================] - 83s 32ms/step - loss: 0.0964 - accuracy: 0.9722
Epoch 10/100
2623/2623 [==============================]

2623/2623 [==============================] - 73s 28ms/step - loss: 0.0854 - accuracy: 0.9788
Epoch 79/100
2623/2623 [==============================] - 73s 28ms/step - loss: 0.0913 - accuracy: 0.9778
Epoch 80/100
2623/2623 [==============================] - 73s 28ms/step - loss: 0.0907 - accuracy: 0.9781
Epoch 81/100
2623/2623 [==============================] - 72s 28ms/step - loss: 0.0931 - accuracy: 0.9782
Epoch 82/100
2623/2623 [==============================] - 73s 28ms/step - loss: 0.0870 - accuracy: 0.9794
Epoch 83/100
2623/2623 [==============================] - 72s 28ms/step - loss: 0.0943 - accuracy: 0.9783
Epoch 84/100
2623/2623 [==============================] - 73s 28ms/step - loss: 0.0935 - accuracy: 0.9784
Epoch 85/100
2623/2623 [==============================] - 79s 30ms/step - loss: 0.0892 - accuracy: 0.9780
Epoch 86/100
2623/2623 [==============================] - 79s 30ms/step - loss: 0.0963 - accuracy: 0.9774
Epoch 87/100
2623/2623 [==============================] - 8

In [44]:
model.save('model_a1_f1_{0}.h5'.format(ep))  # creates a HDF5 file 'my_model.h5'


In [39]:
dataset = fetch_dataset('dataset_g1_slim.pkl')
data_test = make_tf_dataset(dataset, window_size, num_of_features, label_length ).batch(batch_size)
results = model.evaluate(data_test)

    725/Unknown - 7s 9ms/step - loss: 0.6802 - accuracy: 0.8719

In [42]:
dataset = fetch_dataset('dataset_f1_g1_slim.pkl')
data_train = make_tf_dataset(dataset, window_size, num_of_features, label_length ).batch(batch_size)


model.save('model_f1_g1_{0}.h5'.format(ep))  # creates a HDF5 file 'my_model.h5'

model.fit(data_train, epochs=ep)



Epoch 1/100
3008/3008 [==============================] - 110s 36ms/step - loss: 0.5358 - accuracy: 0.9143
Epoch 2/100
3008/3008 [==============================] - 107s 36ms/step - loss: 0.2218 - accuracy: 0.9468
Epoch 3/100
3008/3008 [==============================] - 98s 33ms/step - loss: 0.1800 - accuracy: 0.9538
Epoch 4/100
3008/3008 [==============================] - 122s 41ms/step - loss: 0.1741 - accuracy: 0.9580
Epoch 5/100
3008/3008 [==============================] - 102s 34ms/step - loss: 0.1584 - accuracy: 0.9601
Epoch 6/100
3008/3008 [==============================] - 100s 33ms/step - loss: 0.1601 - accuracy: 0.9617
Epoch 7/100
3008/3008 [==============================] - 97s 32ms/step - loss: 0.1467 - accuracy: 0.9636
Epoch 8/100
3008/3008 [==============================] - 104s 35ms/step - loss: 0.1463 - accuracy: 0.9637
Epoch 9/100
3008/3008 [==============================] - 97s 32ms/step - loss: 0.1353 - accuracy: 0.9654
Epoch 10/100
3008/3008 [=========================

3008/3008 [==============================] - 92s 31ms/step - loss: 0.1201 - accuracy: 0.9750
Epoch 78/100
3008/3008 [==============================] - 93s 31ms/step - loss: 0.1221 - accuracy: 0.9744
Epoch 79/100
3008/3008 [==============================] - 92s 31ms/step - loss: 0.1220 - accuracy: 0.9731
Epoch 80/100
3008/3008 [==============================] - 94s 31ms/step - loss: 0.1146 - accuracy: 0.9750
Epoch 81/100
3008/3008 [==============================] - 88s 29ms/step - loss: 0.1176 - accuracy: 0.9759
Epoch 82/100
3008/3008 [==============================] - 89s 30ms/step - loss: 0.1114 - accuracy: 0.9746
Epoch 83/100
3008/3008 [==============================] - 107s 36ms/step - loss: 0.1096 - accuracy: 0.9762
Epoch 84/100
3008/3008 [==============================] - 105s 35ms/step - loss: 0.1123 - accuracy: 0.9750
Epoch 85/100
3008/3008 [==============================] - 113s 38ms/step - loss: 0.1205 - accuracy: 0.9734
Epoch 86/100
3008/3008 [==============================] 

KeyboardInterrupt: 

In [41]:
dataset = fetch_dataset('dataset_f1_slim.pkl')
data_test = make_tf_dataset(dataset, window_size, num_of_features, label_length ).batch(batch_size)
results = model.evaluate(data_test)

   2283/Unknown - 19s 9ms/step - loss: 2.2402 - accuracy: 0.8191